## Convert PyTorch model to ONNX and OpenVINO IR

In [6]:
import sys
sys.path.append("../utils")
import time
from pathlib import Path

import cv2
import numpy as np
import torch
from IPython.display import Markdown, display
from fastseg import MobileV3Large
from openvino.runtime import Core

from notebook_utils import CityScapesSegmentation, segmentation_map_to_image, viz_result_image

### Settings

In [7]:
IMAGE_WIDTH = 1024  # Suggested values: 2048, 1024 or 512. The minimum width is 512.
# Set IMAGE_HEIGHT manually for custom input sizes. Minimum height is 512
IMAGE_HEIGHT = 1024 if IMAGE_WIDTH == 2048 else 512
DIRECTORY_NAME = "model"
BASE_MODEL_NAME = DIRECTORY_NAME + f"/fastseg{IMAGE_WIDTH}"

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

### Download the Fastseg model

In [8]:
print("Downloading the Fastseg model (if it has not been downloaded before)....")
model = MobileV3Large.from_pretrained().eval()
print("Loaded PyTorch Fastseg model")

# Save the model
model_path.parent.mkdir(exist_ok=True)
torch.save(model.state_dict(), str(model_path))
print(f"Model saved at {model_path}")

Loading pretrained model mobilev3large-lraspp with F=128...
Loaded PyTorch Fastseg model
Model saved at model\fastseg1024.pth


## ONNX model Conversion

### Convert PyTorch model to ONNX

In [9]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)

    # For the Fastseg model, setting do_constant_folding to False is required
    # for PyTorch>1.5.1
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        opset_version=11,
        do_constant_folding=True
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

c:\Users\hokuto\AppData\Local\Programs\Python\Python39\lib\site-packages\geffnet\conv2d_layers.py:47: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((-(i // -s) - 1) * s + (k - 1) * d + 1 - i, 0)


RuntimeError: ONNX export failed: Couldn't export Python operator HardSwishJitAutoFn

Defined at:


Graph we tried to export:
graph(%x.1 : Float(1, 3, 512, 1024, strides=[1572864, 524288, 1024, 1], requires_grad=0, device=cpu),
      %trunk.block2.0.se.conv_reduce.weight : Float(24, 72, 1, 1, strides=[72, 1, 1, 1], requires_grad=1, device=cpu),
      %trunk.block2.0.se.conv_reduce.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %trunk.block2.0.se.conv_expand.weight : Float(72, 24, 1, 1, strides=[24, 1, 1, 1], requires_grad=1, device=cpu),
      %trunk.block2.0.se.conv_expand.bias : Float(72, strides=[1], requires_grad=1, device=cpu),
      %trunk.block2.1.se.conv_reduce.weight : Float(32, 120, 1, 1, strides=[120, 1, 1, 1], requires_grad=1, device=cpu),
      %trunk.block2.1.se.conv_reduce.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %trunk.block2.1.se.conv_expand.weight : Float(120, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu),
  

In [ ]:
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1, 3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
"""